In [0]:
-- result is very large, therefore a subsstring is used for the display
use redox.himss25;

select try_parse_json('{
  "entry": [
    {
      "fullUrl": "https://fhir.redoxengine.com/fhir-sandbox/DiagnosticReport/b5041a91-90c8-4290-85df-2dd73e90d897",
      "resource": {
        "basedOn": [
          {
            "reference": "ServiceRequest/66fac6da-0bb9-4a91-92db-99fd9997c908"
          }
        ],
        "code": {
          "coding": [
            {
              "code": "REPORT",
              "display": "Karius Lab Test Results",
              "system": "http://karius.com/fhir/CodeSystem/custom-codes"
            }
          ]
        },
        "encounter": {
          "reference": "Encounter/f2fa31d8-4d00-4207-9e9c-78e0ae29fccb"
        },
        "id": "b5041a91-90c8-4290-85df-2dd73e90d897",
        "issued": "2025-02-12T18:07:10.080Z",
        "meta": {
          "lastUpdated": "2025-02-27T18:58:35.799180+00:00",
          "versionId": "MTc0MDY4MjcxNTc5OTE4MDAwMA"
        },
        "presentedForm": [
          {
            "contentType": "application/pdf",
            "creation": "2025-02-12T18:07:10.080Z",
            "data": "JVBERi0xLjUKJeLjz9MKCjEgMCBvYmoKP"
          }
        ]
      }
    }
  ]
}') as example;

with eg as (
  SELECT
    "post" as http_method
    ,"DiagnosticReport" as resource
    ,"_search" as action
    ,"subject=Patient/81c2f5eb-f99f-40c4-b504-59483e6148d7" as data
)
,response as (
  SELECT
    try_parse_json(redox_request(
    http_method
    ,resource
    ,action
    ,data
  )) as response
FROM 
  eg
)
SELECT
  entry.value:fullUrl::string as fullUrl
  ,entry.value:resource.based_on
  ,entry.value:resource.code
  ,entry.value:resource.encounter.reference::string as encounter_url
  ,entry.value:resource.id::string as diagnostic_report_id
  ,entry.value:resource.issued::timestamp as issued
  ,entry.value:resource.meta
  ,presentedForm.value:contentType::string as content_type
  ,presentedForm.value:creation::timestamp as creation
  ,unbase64(presentedForm.value:data::string) as diagnostic_report
FROM 
  response 
  ,LATERAL variant_explode(response:entry) as entry
  ,LATERAL variant_explode(entry.value:resource.presentedForm) as presentedForm
;